In [34]:
import requests as rq 
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
# from plot import do_plot
from bs4 import BeautifulSoup as bs
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression

import seaborn as sns
from pylab import savefig
import io

# FLASK
# !pip install flask-ngrok
!pip install flask==0.12.2
!pip install -U flask-cors

# from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, make_response, send_file
from flask_cors import CORS, cross_origin

app = Flask(__name__)
cors = CORS(app,resources={r"/": {"origins": "http://localhost:3000/"}})
app.config['SECRET_KEY'] = 'the quick brown fox jumps over the lazy dog'
app.config['CORS_HEADERS'] = 'Content-Type'
# run_with_ngrok(app)   #starts ngrok when the app is run

Requirement already up-to-date: flask-cors in c:\users\haris\anaconda3\lib\site-packages (3.0.8)


In [35]:
# CRAWLING
# salah satu contoh raw json yg bakal diambil
# {"bodystyleName":"Crew Cab Pickup","sellerId":424805,"customerId":4083,"isCpoVehicle":false,"price":47193,"listingId":784826056,"mkId":20053,"mkNm":"Chevrolet","mdId":22061,"mdNm":"Silverado 1500","trimId":24332,"trimName":"LT","modelYearId":36620293,"modelYear":2020,"stkTyp":"New","state":"CT","zipcode":"11430"}

def getLiData(ul_tag_parent, ul_selector, ul_selector_name, li_selector = None, li_selector_name = None):
  ul_data = ul_tag_parent.find("ul",{ul_selector:ul_selector_name})
  raw_li = ul_data.findAll("li")

  if li_selector != None:
    raw_li = ul_data.findAll("li",{li_selector:li_selector_name})

  li_data = {}
  for li in raw_li:
    txt = li.text.strip()
    value = txt[txt.index(':'):]
    key = txt[:txt.index(':')].lower().replace(". ","_")
    value = value.replace(":\n","").lstrip()
    li_data[key] = value
  
  # renaming the key sm dgn pop the value and assign it to the knew key
  li_data["transmission"]

  return li_data

def getJsonData(tag_parent, tag_type, selector, selector_name, attribute = ""):
  details = tag_parent.find("div",{"class":"listing-row__details"})
  raw_json = json.loads(details.find(tag_type,{selector:selector_name})[attribute])
  return raw_json

def filterJsonData(data ,**kwargs):
  filtered_data = {}
  for key,value in kwargs.items():
        filtered_data[key] = data[value]
  return filtered_data

def getPhoto(tag_parent):
  photo_container = tag_parent.find("div",{"class":"listing-row__photo-container"})
  img_raw = str(photo_container.select_one(".photo-image"))
  img_raw= img_raw[img_raw.index('('):img_raw.index(')')].replace("(","")
  image = {
      "image": img_raw
  }

  return image

selected_field = {
    "body_name": "bodystyleName",
    "price":"price",
    "make":"mkNm",
    "model":"mdNm",
    "trim_name":"trimName",
    "year":"modelYear"
}


page_ke = 1
url = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=localOnly&mkId=20005,20053,20015,20017,20028,20077,20088&page=' + str(page_ke) + '&perPage=100&rd=99999&searchSource=PAGINATION&sort=relevance&stkTypId=28881&zc=11430'
temp_raw = []
while len(temp_raw) < 10:
  page = rq.get(url)
  # print(page.status_code) #cek status request, jika 200 maka berhasil masuk
  soup = bs(page.content, 'lxml')
  listings = soup.find("div",{"class": "shop-srp-listings"})
  batch = [
            {
                **{
                    # "href": "cars.com"+ container.find("a",{"class": "shop-srp-listings__listing"},href=True)["href"],
                   "mileage": container.find("span",{"class":"listing-row__mileage"}).text.strip()
                  #  background-image: url(https://www.cstatic-images.com/phototab/in/v1/449520/1FM5K8D80HGA31276/0c030ab7f4ff22fe334302bc95d4f08a.jpg);
                   }
             ,
             **filterJsonData(getJsonData(container,"div","class","listing-row__save",attribute = "vehicle"),**selected_field),
             **getLiData(container,"class","listing-row__meta"),
             **getPhoto(container)
            }

            for container in listings.findAll("div",{"class":"shop-srp-listings__listing-container"})
            ]

  temp_raw.append(batch)
  print(batch[page_ke-1])
  page_ke += 1
  url = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=localOnly&mkId=20005,20053,20015,20017,20028,20077,20088&page=' + str(page_ke) + '&perPage=100&rd=99999&searchSource=PAGINATION&sort=relevance&stkTypId=28881&zc=11430'
  

raw_data = []
for raw in temp_raw:
  raw_data += raw
dataset = pd.DataFrame.from_dict(raw_data)
dataset.to_csv("raw_data.csv")
print("CRAWLING DONE")

{'mileage': '8,070 mi.', 'body_name': 'SUV', 'price': 69800, 'make': 'BMW', 'model': 'X7', 'trim_name': 'xDrive40i', 'year': 2019, 'ext_color': 'White', 'int_color': 'Other', 'transmission': 'Automatic', 'drivetrain': 'AWD', 'image': 'https://www.cstatic-images.com/phototab/in/v1/421976/5UXCW2C5XKL080874/275a45c13917ac61670edd9ae6d5a17c.jpg'}
{'mileage': '76,502 mi.', 'body_name': 'SUV', 'price': 5895, 'make': 'Honda', 'model': 'Element', 'trim_name': 'EX', 'year': 2004, 'ext_color': 'Blue', 'int_color': 'Gray', 'transmission': 'Automatic', 'drivetrain': '4x4/4-wheel drive', 'image': 'https://www.cstatic-images.com/phototab/in/v1/426261/5J6YH28594L011553/5ce9b0c5748ecb684c527eefe55c6791.jpg'}
{'mileage': '23,727 mi.', 'body_name': 'Sedan', 'price': 8900, 'make': 'Nissan', 'model': 'Sentra', 'trim_name': 'S', 'year': 2018, 'ext_color': 'White', 'int_color': 'Black', 'transmission': 'CVT', 'drivetrain': 'FWD', 'image': 'https://www.cstatic-images.com/phototab/in/v1/33005319/3N1AB7AP2JY33

In [ ]:
# PREPROCESSING 1
def replace_nan():
  global dataset
  # change a pure empty string ("") into NaN
  dataset.replace("",np.nan,inplace=True)
  # mengganti unknown yg ada di transmission & drivetrain menjadi modus
  dataset["transmission"].replace(r"Unknown",np.nan, regex=True, inplace=True)
  dataset["drivetrain"].replace("Unknown",np.nan, inplace=True)
  # dataset["ext_color"].replace("Other",np.nan, inplace=True)
  # dataset["int_color"].replace("Other",np.nan, inplace=True)
  # pakai method forward fill
  dataset.fillna(method = "ffill", inplace= True)
  dataset["mileage"].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
  dataset["mileage"] = dataset["mileage"].astype(int)
    
    
    

replace_nan()
clean_data = dataset.copy()
ori_col_names = dataset.loc[:, dataset.columns != "price"].columns
xytrain_xytest = []
ytest_ypred = []
model_score = 0



# PREPROCESSING 2
def convert_to_numerical(categorical_col, modes_val):
  global dataset
  # One Hot Encoder
  cat_modes_col = list(zip(categorical_col, modes_val))
  df = []
  for category_name, mode_val in cat_modes_col:
    # top10 = dataset[category_name].value_counts()[:10].index
    # dataset.loc[~dataset[category_name].isin(top10), category_name] = 'Other'
    
    dataset[category_name] = pd.Categorical(dataset[category_name])
    df_dummies = pd.get_dummies(dataset[category_name], prefix=category_name)
    # drop a dummy column
    dummy_col_name = category_name + "_"+mode_val
    if dummy_col_name in df_dummies.columns:
      df_dummies.drop(columns=dummy_col_name, inplace=True)

    dataset = pd.concat([dataset, df_dummies], axis=1)

def get_each_modes(df,categorical_col):
  modes_val = [df[col_name].mode()[0] for col_name in categorical_col]
  return modes_val

def tts(df):
# TRAIN_TEST_SPLIT
  X = df.loc[:, df.columns != "price"].values
  y = df["price"].values
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

  return [X_train, y_train, X_test, y_test]

def train_test_dataset(datasets, col_names):
  X_train, y_train, X_test, y_test = datasets
  test_dataset = pd.DataFrame(X_test[:,:11], columns= col_names)
  test_dataset["price"] = y_test
  test_dataset["predicted_price"] = "-"
  test_dataset.to_csv("test_dataset.csv")

  train_dataset = pd.DataFrame(X_train[:,:11], columns= col_names)
  train_dataset["price"] = y_train
  train_dataset.to_csv("train_dataset.csv")

  return test_dataset

# MAKE MODEL
def make_model(xytrain_test_data):
  X_train, y_train, X_test, y_test = xytrain_test_data 
  reg = LinearRegression()  
  reg.fit(X_train, y_train)
  reg_score = reg.score(X_test, y_test)
  print("score: " + str(reg_score * 100) + "%")
  print("--------------------------------------------")

  # Prediction on test data
  y_pred = reg.predict(X_test)
  y_test_train = (y_test, y_pred)
  result = (reg_score, y_test_train)
  return result
  
def make_plot(y_test_train):

  y_test, y_pred = y_test_train
  bytes_image = io.BytesIO()  
  df_target = pd.DataFrame({'price': y_test, 'predicted_price': y_pred})  
  df_target.head(25).plot(kind='bar',figsize=(10,8))
  plt.grid(which='major', linestyle='-',  linewidth='0.5', color='black')
  plt.grid(which='minor', linestyle=':', linewidth='0.5', color='yellow')
  plt.savefig(bytes_image, format="png")
  bytes_image.seek(0)
  return bytes_image

def make_plot_seaborn(y_test_train):
  y_test,y_pred = y_test_train
  bytes_image = io.BytesIO()
  df_target = pd.DataFrame({'price': y_test, 'predicted_price': y_pred}) 
  sns_plot = sns.lmplot(x='predicted_price',y='price',data=df_target,fit_reg=True)
  sns_plot.savefig(bytes_image, format="png")
#   fig = sns_plot.get_figure()
# #   fig.savefig(bytes_image)
#   fig.savefig(bytes_image, format="png")
  bytes_image.seek(0)
  return bytes_image

def get_best_prediction():
  global ytest_ypred
  yt, yp = ytest_ypred
  ytp = list(zip(yt,yp))
  print(ytp)
  print(ytp[0])
  list_best_pred = [[data1,data2] for data1,data2 in ytp if -1000 <= data1-data2 <= 1000]
  return list_best_pred


@app.route("/")
@cross_origin(origin='localhost:3000',headers=['Content-Type'])
# @cross_origin(allow_headers=['Content-Type'])
def homepage():
  global clean_data
  clean_data.to_csv("clean_data.csv")
  # response = make_response()
  response = jsonify(clean_data.to_dict("records"))
  # response.headers.add("Access-Control-Allow-Headers", "*")
  # response.headers.add('Access-Control-Allow-Origin', '*')
  # response.headers.add("Access-Control-Allow-Origin", "*")
  print(response)
  return response


# def json_response(payload, status=200):
#      return (json.dumps(payload), status, {'content-type': 'application/json'})

@app.route("/train-test")
@cross_origin(origin='localhost:3000',headers=['Content-Type'])
# @cross_origin(allow_headers=['Content-Type'])
def train_test():
  global xytrain_xytest
  categorical_col = ["body_name","make", "model", "trim_name", "ext_color", "int_color", "transmission", "drivetrain"]
  # drop mode dari setiap kategorical column (untuk menghindari dummy variable trap)
  modes_val = get_each_modes(dataset, categorical_col)
  convert_to_numerical(categorical_col, modes_val)
  dataset.to_csv("numerical_dataset.csv")
  xytrain_xytest_temp = tts(dataset)
  # return an array
  xytrain_xytest = [data for data in xytrain_xytest_temp]
  train_dataset =  train_test_dataset(xytrain_xytest_temp, ori_col_names)
  response = jsonify(train_dataset.to_dict("records"))
  print(response)
  return response

@app.route("/predict")
@cross_origin(origin='localhost:3000',headers=['Content-Type'])
def predict():
# delete unused columns in X datasets
  global xytrain_xytest, ytest_ypred, model_score
  X_train, y_train, X_test, y_test = xytrain_xytest

  X_train_image = X_train[:,3]
  X_train_mileage = X_train[:,6]
  X_train_rest = X_train[:,10:]
  X_train_temp = list(zip(X_train_mileage,X_train_rest))
  # mileage, year, dummies_var...
  X_train_new = [ np.insert(rest, 0, mile)  for mile, rest in X_train_temp ]

  X_test_image = X_test[:,3]
  X_test_mileage = X_test[:,6]
  X_test_rest = X_test[:,10:]
  X_test_temp = list(zip(X_test_mileage,X_test_rest))
  X_test_new = [ np.insert(rest, 0, mile)  for mile, rest in X_test_temp ]

  xytrain_xytest = [X_train_new, y_train, X_test_new, y_test]
  model_score, model_result = make_model(xytrain_xytest)
  ytest_ypred = [data for data in model_result]
  ytrain_pred_temp = list(zip(model_result[0],model_result[1]))
    
  test_dataset = pd.read_csv("test_dataset.csv").iloc[:,1:]
  test_dataset["predicted_price"] = model_result[1]
  test_dataset.to_csv("test_dataset_final.csv")
  response = jsonify(test_dataset.to_dict("records"))
  print(response)
  return response

@app.route("/result")
@cross_origin(origin='localhost',headers=['Content-Type'])
def result():
  global model_score
  score = model_score * 100
  keterangan = ""
  kesimpulan = ""
  if score < 65:
    keterangan = "Buruk"
    kesimpulan = "Model yang dibuat belum memadai untuk dataset ini"
  elif 65 <= score >= 75:
    keterangan = "Cukup Baik"
    kesimpulan = "Model yang dibuat cukup memadai untuk dataset ini"
  elif 75 <= score >= 80:
    keterangan = "Baik"
    kesimpulan = "Model yang dibuat memadai untuk dataset ini"
  else:
    keterangan = "Sangat Baik"
    kesimpulan = "Model yang dibuat sangat memadai untuk dataset ini"
    
# return array with 1 row,2 col
#   bst_pred = get_best_prediction()
#   if len(bst_pred) <= 0:
#     bst_pred = ["-"]
  dict_resp = dict({"score": score,"keterangan": keterangan, "kesimpulan": kesimpulan})
  response = jsonify(dict_resp)
  print(response)
  return response

@app.route("/plot/plt")
@cross_origin(origin='localhost',headers=['Content-Type'])
def send_plot():
  global ytest_ypred
#   ytest_ypred = two d list
  plot_result = make_plot(ytest_ypred)
  return send_file(plot_result, attachment_filename="plot.png", mimetype="image/png")

@app.route("/plot/seaborn")
@cross_origin(origin='localhost',headers=['Content-Type'])
def send_plot_seaborn():
  global ytest_ypred
  plot_result =  make_plot_seaborn(ytest_ypred)
  return send_file(plot_result, attachment_filename="seaborn_plot.png", mimetype="image/png")


if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2019 21:22:09] "GET / HTTP/1.1" 200 -


<Response 416292 bytes [200 OK]>


127.0.0.1 - - [10/Dec/2019 21:22:15] "GET / HTTP/1.1" 200 -
[2019-12-10 21:22:15,335] ERROR in app: Exception on /plot/seaborn [GET]
Traceback (most recent call last):
  File "C:\Users\Haris\Anaconda3\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Haris\Anaconda3\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Haris\Anaconda3\lib\site-packages\flask_cors\extension.py", line 161, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\Haris\Anaconda3\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Haris\Anaconda3\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\Haris\Anaconda3\lib\site-packages\flask\app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  

<Response 416292 bytes [200 OK]>
<Response 113 bytes [200 OK]>


127.0.0.1 - - [10/Dec/2019 21:22:19] "GET /train-test HTTP/1.1" 200 -


...

<Response 133644 bytes [200 OK]>


127.0.0.1 - - [10/Dec/2019 21:28:40] "GET /predict HTTP/1.1" 200 -


score: 69.6110491438517%
--------------------------------------------
<Response 137993 bytes [200 OK]>


127.0.0.1 - - [10/Dec/2019 21:28:43] "GET /result HTTP/1.1" 200 -


<Response 135 bytes [200 OK]>
